In [1]:
import copy
import random
from enum import Enum

In [2]:
#スコーレム関数のクラス
class skolem_func:
    def __init__(self, literal, values):
        self.literal = literal
        self.values = values
    def __eq__(self, other):
        if type(other) is not skolem_func:
            return False
        if self.literal == other.literal \
        and self.values == other.values:
            return True
        else:
            return False
    def assign(self, key, value):
        for i,v in enumerate(self.values):
            if type(v) is str:
                if key == v: self.values[i] = value
            elif type(v) is skolem_func:
                v.assign( key, value)
    def __str__(self):
        l = " "
        l += self.literal
        l += "("
        for value in self.values:
            l += f" {value},"
        l = l[:-1]
        l += ")"
        return l
    
#節クラス，リテラル，変数，正負を持つ
class Clause:
    def __init__(self, literal, values, pos = True):
        self.pos = pos
        self.literal = literal
        self.values = values
    def __lt__(self, other):
        return len(self.values) < len(other.values)
    def __eq__(self, other):
        if type(other) is not Clause:
            return False
        if self.pos == other.pos \
        and self.literal == other.literal \
        and self.values == other.values:
            return True
        else:
            return False
    def __str__(self):
        l = ""
        if not self.pos:
            l += "! "
        l += self.literal
        l += "("
        for value in self.values:
            l += f" {value},"
        l = l[:-1]
        l += ")"
        return l
    def assign(self, key, value):
        for i,v in enumerate(self.values):
            if type(v) is str:
                if key == v: self.values[i] = value
            elif type(v) is skolem_func:
                v.assign(key, value)
    def neg(self):
        cl = self.copy()
        cl.pos =  not cl.pos
        return cl
    def copy(self):
        cl = copy.copy(self)
        return cl

#節集合（リストの継承）
class ClauseSet(list):
    def __init__(self, *args, **kw):
        list.__init__(self, *args, **kw)
    def __it__(self):
        return len(self)<len(self)
    def __str__(self):
        l = "{"
        if not self:
            l += "}"
            return l
        for clause in self:
            l += " "
            l += f"{clause}"
            l += ","
        l = l[:-1]
        l += "}"
        return l

#MGUクラス（リストの継承）
class Mgu(list):
    def __init__(self, *args, **kw):
        list.__init__(self, *args, **kw)
    def __str__(self):
        l = ""
        if not self:
            l += "{}"
            return l
        for g in self:
            l += " "
            l += f"{{{g[0]}/{g[1]}}}"
            l += ","
        l = l[:-1]
        return l

#mguの代入処理，右と左
class AssignType(Enum):
    LEFT = 0
    RIGHT = 1
    ERROR = 2

In [3]:
#２つの節集合から矛盾する組を消す
def resolution(clauseSet1, clauseSet2):
    mgu = Mgu([])
    Set1 = copy.deepcopy(clauseSet1)
    Set2 = copy.deepcopy(clauseSet2)
    Updated = True
    n = 0
    while(Updated and n<10):
        n+=1
        Updated = False
        for i, _ in enumerate(Set1):
            for j, _ in enumerate(Set2):
                if Set1[i].literal == Set2[j].literal:
                    isUpdate = mguUpdate(i, j, Set1, Set2, mgu)
                    if isUpdate:
                        Updated = True
                    break
            if Updated:
                break
    return removeConflict(Set1, Set2), mgu

#mguの更新
def mguUpdate(i, j, Set1, Set2, mgu):
    Updated = False
    for val1,val2 in zip(Set1[i].values, Set2[j].values):
        assignType = getAssignType(val1, val2)
        if assignType is AssignType.LEFT:
            assingCaluseSet(Set1, val1, val2)
            mguAssign(mgu, val1, val2)
            mgu.append([val1,val2])
            Updated = True
        elif assignType is AssignType.RIGHT:
            assingCaluseSet(Set2, val2, val1)
            mguAssign(mgu, val2, val1)
            mgu.append([val2,val1])
            Updated = True
    return Updated

#mguの代入
def mguAssign(mgu, key, value):
    for i,_ in enumerate(mgu):
        if type(mgu[i][0]) is str:
            if key == mgu[i][0]:
                mgu[i][0] = value
        elif type(mgu[i][0]) is skolem_func:
            mgu[i][0].assign(key, value)
        if type(mgu[i][1]) is str:
            if key == mgu[i][1]:
                mgu[i][1] = value
        elif type(mgu[i][1]) is skolem_func:
            mgu[i][1].assign(key, value)

#節集合へ代入
def assingCaluseSet(clauseSet, key, value):
    for clause in clauseSet:
        clause.assign(key, value)

#代入方法
def getAssignType(val1, val2):
    if type(val1) is str and val1 in values_list and val1 != val2:
        return AssignType.LEFT
    elif type(val2) is str and val2 in values_list and val1 != val2:
        return AssignType.RIGHT
    elif type(val1) is skolem_func and type(val2) is skolem_func:
        if val1.literal  == val2.literal:
            for v1, v2 in zip(val1.values, val2.values):
                if v1 != v2:
                    return getAssignType(v1, v2)
                else: return AssignType.ERROR
    else: return  AssignType.ERROR

#矛盾の解消
def removeConflict(clauseSet1, clauseSet2):
    Set1 = clauseSet1.copy()
    Set2 = clauseSet2.copy()
    random.shuffle(Set1)
    random.shuffle(Set2)
    clauseSet = ClauseSet()
    for i, _ in enumerate(Set1):
        isContradict = False
        for j, _ in enumerate(Set2):
            if clauseContradict(Set1[i], Set2[j]):
                isContradict = True
                Set1.pop(i)
                Set2.pop(j)
                break
        if isContradict:
            break
    clauseSet.extend(Set1)
    clauseSet.extend(Set2)
    return clauseSet

#矛盾
def clauseContradict(clause1, clause2):
    return clause1 == clause2.neg()

#最大の要素を持つ節
def maxGroup(group):
    t = 0
    for c in group:
        t= max(len(c),t)
    return t

In [4]:
#融合法によって問題を解く
def solveClauseSet(values_list,clauseGroup):
    clauseGroup.extend(psi_neg)
    isSolved = False
    ansLists = [None]*len(clauseGroup)
    mguLists = [None]*len(clauseGroup)
    resolutionCount = 0
    while(not isSolved):
        clauseSets = []
        ansSets = []
#         print(resolutionCount)
        resolutionCount+=1
        for i,clauseSet1 in enumerate(clauseGroup):
            for j,clauseSet2 in enumerate(clauseGroup[i+1:]):
                clauseSet, mgu = resolution(clauseSet1, clauseSet2)
                if not clauseSet:
                    isSolved = True
                    clauseSets.append(clauseSet)
                    ansSets.append([clauseSet1, clauseSet2, mgu])
                    print("solved")
                    break
                if (len(clauseSet) == len(clauseSet1)+len(clauseSet2)-2 \
                or len(clauseSet)<= maxGroup(clauseGroup))\
                and clauseSet not in clauseSets and clauseSet not in clauseGroup:
                    clauseSets.append(clauseSet)
                    ansSets.append([clauseSet1, clauseSet2, mgu])
            if isSolved:
                break
        if isSolved:
                for clauseSet, ansSet in zip(clauseSets, ansSets):
                    if clauseSet not in clauseGroup:
                        clauseGroup.append(clauseSet)
                        ansLists.append(ansSet)
                break
#         print([clauseSet in clauseGroup for clauseSet in clauseSets])
        if not ClauseSet or all([clauseSet in clauseGroup for clauseSet in clauseSets]):
            print("unsolved")
            break
        for clauseSet, ansSet in zip(clauseSets, ansSets):
            if clauseSet not in clauseGroup:
                clauseGroup.append(clauseSet)
                ansLists.append(ansSet)
    return ansLists

In [5]:
#得られた推論で用いたを節を得る
def findResolution(ans, ansLists, clauseGroup):
    for i,_ in enumerate(clauseGroup):
        if ans == clauseGroup[i]:
            if ansLists[i] is not None:
                findResolution(ansLists[i][0],ansLists,clauseGroup)
                findResolution(ansLists[i][1],ansLists,clauseGroup)
                print(f"resolution:\n\t{ansLists[i][0]} and {ansLists[i][1]}\n\tmgu:{ansLists[i][2]}\n\t->{clauseGroup[i]}\n")
            break

In [6]:
#推論の過程を出力
def printResult(ansLists):
    if ansLists[-1][0]:
        findResolution(ansLists[-1][0],ansLists,clauseGroup)
    if ansLists[-1][1]:
        findResolution(ansLists[-1][1],ansLists,clauseGroup)

    print(f"resolution:\n\t{ansLists[-1][0]} and {ansLists[-1][1]}\n\tmgu:{ansLists[-1][2]}\n\t->{clauseGroup[-1]}")

In [7]:
#問題-sample
values_list = ['x','y','z']
clauseGroup = [ClauseSet([Clause('Patient', [skolem_func('f',['y'])])]),
               ClauseSet([Clause('Doctor', ['y'], pos = False),
                Clause('Like',[skolem_func('f', ['y']), 'y'])]),
               ClauseSet([Clause('Patient', ['x'], pos = False),
                Clause('FakeDoctor', ['z'], pos = False),
                Clause('Like', ['x','z'], pos = False)]),
               ClauseSet([Clause('Patient', ['a'])])]
psi_neg = [ClauseSet([Clause('Doctor',['y'])]),ClauseSet([Clause('FakeDoctor',['z'])])]
clauseGroup.extend(psi_neg)
ansLists=solveClauseSet(values_list,clauseGroup)
printResult(ansLists)

solved
resolution:
	{ Patient(  f( y))} and { ! Patient( x), ! FakeDoctor( z), ! Like( x, z)}
	mgu: {x/ f( y)}
	->{ ! Like(  f( y), z), ! FakeDoctor( z)}

resolution:
	{ ! Doctor( y), Like(  f( y), y)} and { Doctor( y)}
	mgu:{}
	->{ Like(  f( y), y)}

resolution:
	{ ! Like(  f( y), z), ! FakeDoctor( z)} and { Like(  f( y), y)}
	mgu: {z/y}
	->{ ! FakeDoctor( y)}

resolution:
	{ FakeDoctor( z)} and { ! FakeDoctor( y)}
	mgu: {z/y}
	->{}


In [8]:
#問2ステップ1
values_list = []
clauseGroup = [ClauseSet([Clause("True", "A", pos = False),Clause("True", "C")]),
               ClauseSet([Clause("True", "A"),Clause("True", "C", pos = False)]),
               ClauseSet([Clause("True", "A", pos = False),Clause("True", "B", pos = False)]),
               ClauseSet([Clause("True", "A"),Clause("True", "B")]),
               ClauseSet([Clause("True", "A", pos = False),Clause("True", "B", pos = False),Clause("True", "C", pos = False)]),
               ClauseSet([Clause("True", "A", pos = False),Clause("True", "B"),Clause("True", "C")]),
               ClauseSet([Clause("True", "A"),Clause("True", "B", pos = False),Clause("True", "C")]),
               ClauseSet([Clause("True", "A"),Clause("True", "B"),Clause("True", "C", pos = False)]),
              ]
psi_neg = [ClauseSet([Clause("True", "A", pos = False)])]
clauseGroup.extend(psi_neg)

ansLists=solveClauseSet(values_list,clauseGroup)
printResult(ansLists)

solved
resolution:
	{ True( A), ! True( C)} and { ! True( A)}
	mgu:{}
	->{ ! True( C)}

resolution:
	{ True( A), True( B)} and { ! True( A)}
	mgu:{}
	->{ True( B)}

resolution:
	{ True( A), ! True( B), True( C)} and { ! True( A)}
	mgu:{}
	->{ True( C), ! True( B)}

resolution:
	{ True( B)} and { True( C), ! True( B)}
	mgu:{}
	->{ True( C)}

resolution:
	{ ! True( C)} and { True( C)}
	mgu:{}
	->{}


In [9]:
#問2ステップ2
values_list = []
clauseGroup = [ClauseSet([Clause("True", "A", pos = False),Clause("True", "C")]),
               ClauseSet([Clause("True", "A"),Clause("True", "C", pos = False)]),
               ClauseSet([Clause("True", "A", pos = False),Clause("True", "B", pos = False)]),
               ClauseSet([Clause("True", "A"),Clause("True", "B")]),
               ClauseSet([Clause("True", "A", pos = False),Clause("True", "B", pos = False),Clause("True", "C", pos = False)]),
               ClauseSet([Clause("True", "A", pos = False),Clause("True", "B"),Clause("True", "C")]),
               ClauseSet([Clause("True", "A"),Clause("True", "B", pos = False),Clause("True", "C")]),
               ClauseSet([Clause("True", "A"),Clause("True", "B"),Clause("True", "C", pos = False)]),
               ClauseSet([Clause("True", "A")])
              ]
psi_neg = [ClauseSet([Clause("True", "A", pos = False),Clause("True", "B" ),Clause("True", "C", pos = False)])]

clauseGroup.extend(psi_neg)
ansLists=solveClauseSet(values_list,clauseGroup)
printResult(ansLists)

solved
resolution:
	{ ! True( A), True( C)} and { True( A)}
	mgu:{}
	->{ True( C)}

resolution:
	{ ! True( A), ! True( B)} and { True( A)}
	mgu:{}
	->{ ! True( B)}

resolution:
	{ True( A)} and { ! True( A), True( B), ! True( C)}
	mgu:{}
	->{ True( B), ! True( C)}

resolution:
	{ ! True( B)} and { True( B), ! True( C)}
	mgu:{}
	->{ ! True( C)}

resolution:
	{ True( C)} and { ! True( C)}
	mgu:{}
	->{}


In [ ]:
#本来解くべき問題だが求められない
values_list = []
clauseGroup = [ClauseSet([Clause("True", "A", pos = False),Clause("True", "C")]),
               ClauseSet([Clause("True", "A"),Clause("True", "C", pos = False)]),
               ClauseSet([Clause("True", "A", pos = False),Clause("True", "B", pos = False)]),
               ClauseSet([Clause("True", "A"),Clause("True", "B")]),
               ClauseSet([Clause("True", "A", pos = False),Clause("True", "B", pos = False),Clause("True", "C", pos = False)]),
               ClauseSet([Clause("True", "A", pos = False),Clause("True", "B"),Clause("True", "C")]),
               ClauseSet([Clause("True", "A"),Clause("True", "B", pos = False),Clause("True", "C")]),
               ClauseSet([Clause("True", "A"),Clause("True", "B"),Clause("True", "C", pos = False)])
              ]
psi_neg = [ClauseSet([Clause("True", "A", pos = False),Clause("True", "B" ),Clause("True", "C", pos = False)])]

clauseGroup.extend(psi_neg)
ansLists=solveClauseSet(values_list,clauseGroup)
printResult(ansLists)